In [11]:
# current 10
# rows: 393 836

In [1]:
# step 1
import pandas as pd
df = pd.read_pickle("dataset.pkl")

In [ ]:
# Remove already processed
import pandas as pd
preprocessed = pd.read_pickle("preprocessed.pkl")
ids = preprocessed['id']
df = df[df['id'] not in ids]

In [2]:
# step 2.1
# Remove duplicates by specific column 
df = df.drop_duplicates(subset=['id'])
# Resetting the DataFrame index
df = df.reset_index()
df = df.drop(columns=['index'])

In [3]:
# step 2.2
# Remove duplicates by specific column 
df = df.drop_duplicates(subset=['text'])
# Resetting the DataFrame index
df = df.reset_index()
df = df.drop(columns=['index'])

In [4]:
# SKIP THIS STEP
# step 3
from nltk import tokenize

# Split text into sentences and assaign each sentence as a new row
id, text, date, likes = [], [], [], []
toDelete = []
for index, row in df.iterrows():
    sentences = tokenize.sent_tokenize(row['text'])
    if len(sentences) > 1:
        toDelete.append(index)
        for i in sentences:
            id.append(index)
            text.append(i)
            date.append(row['date'])
            likes.append(row['likes'])
            
df_new = pd.DataFrame()
df_new['id'] = id
df_new['text'] = text
df_new['date'] = date
df_new['likes'] = likes

# Delete rows from df having multiple sentences
df = df.drop(toDelete)

# Combine two dataframes
concatenated = pd.concat([df, df_new])
# Reset index on concatenated
concatenated = concatenated.reset_index()
concatenated = concatenated.drop(columns=['index'])
df = concatenated

In [4]:
# step 4
# Detect emojis
import emoji
import re

def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

def count_remove_emojis(s):
    return len(''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])), emoji.demojize(s, delimiters=("", ""))
    
emojis = []
without_emojis = []
for text in df['text']:
    len_emojis, new_text = count_remove_emojis(text)
    emojis.append(len_emojis)
    without_emojis.append(new_text)
    
df['clean'] = without_emojis

In [5]:
# step 5
import re
import contractions
import splitter
from autocorrect import Speller
from nltk.tokenize import sent_tokenize, word_tokenize
import enchant
d = enchant.Dict("en_US")
spell = Speller(lang='en')

# Cleaning
def clean_tweet(text):
    # to lower case
    text = text.lower()
    # remove tagged users
    text = re.sub("@[A-Za-z0-9_]+","", text)
    # remove retweet characters
    text = re.sub("rt : ","", text)
    # remove links
    text = re.sub(r'http\S+', '', text)
    # replace new lines with blank space
    text = re.sub(r"\n", " ", text)
    text = re.sub(' +', ' ', text)
    # remove &amp;
    text = re.sub('&amp;', ' ', text)
    # remove special characters
    text = re.sub("[-|(%$)).,#*“”`':;""]", "", text)
    # remove numbers
    text = re.sub(r'[0-9]', '', text)
    # Simplify the emoji text, with removing the '_'
    text = re.sub("_"," ", text)
    return text


# Contractions
def decontract(text):
    # to lower case
    text = text.lower()
    # remove tagged users
    text = re.sub("@[A-Za-z0-9_]+","", text)
    # remove retweet characters
    text = re.sub("rt : ","", text)
    # remove links
    text = re.sub(r'http\S+', '', text)
    # replace new lines with blank space
    text = re.sub(r"\n", " ", text)
    text = re.sub(' +', ' ', text)
    # remove &amp;
    text = re.sub('&amp;', ' ', text)
    
    expanded_words = []
    for word in text.split():
      # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))  
        text = ' '.join(expanded_words)
    return text

clean = []
for i in df['clean']:
    cleaned = decontract(i)
    clean.append(clean_tweet(cleaned))
df['clean'] = clean

In [6]:
# step 6
# Remove non-asci characters
cleaned_ascii = []
for text in df['clean']:
    string_encode = text.encode("ascii", "ignore")
    string_decode = string_encode.decode()
    cleaned_ascii.append(string_decode)
df['clean'] = cleaned_ascii

In [7]:
# step 7.1
# Translate slangs
import pickle
import enchant
d = enchant.Dict("en_US")
        
with open('slang_dictionary.pkl', 'rb') as f:
    slang_dictionary = pickle.load(f)

cleaned_stop = []
for text in df['clean']:
    
    text_tokens = word_tokenize(text)
    
    tokens_without_slang = []
    for token in text_tokens:
        if token.upper() in slang_dictionary.keys() and d.check(token) is not True:
            tokens_without_slang.append(slang_dictionary[token.upper()])
        else: tokens_without_slang.append(token)
    tokens_without_slang = " ".join(tokens_without_slang)
    cleaned_stop.append(tokens_without_slang)

df['clean'] = cleaned_stop

In [8]:
# step 7.2

# Translate negations 
from nltk.tokenize import sent_tokenize, word_tokenize

# First create the dictionary below
with open('negations_dictionary.pkl', 'rb') as f:
    negations_dictionary = pickle.load(f)
    
cleaned_negations = []
for text in df['clean']:
    
    text_tokens = word_tokenize(text)
    
    tokens_without_negations = []
    for idx, token in enumerate(text_tokens):
        if token in negations_dictionary.keys() and idx != 0:
            if text_tokens[idx-1] == 'not':
                tokens_without_negations.append(negations_dictionary[token])
        elif token != 'not': tokens_without_negations.append(token)
            
    tokens_without_negations = " ".join(tokens_without_negations)
    
    cleaned_negations.append(tokens_without_negations)

df['clean'] = cleaned_negations

In [27]:
# Remove punctation
def remove_punctation(text):
    text = re.sub(r'[^\w\s]', '', text)
    # to lower case
    text = text.lower()
    return text

clean = []
for i in df['clean']:
    clean.append(remove_punctation(i))
df['clean'] = clean

In [10]:
# step 7.3
# Extract stop words

# import gensim
# all_stopwords = gensim.parsing.preprocessing.STOPWORDS

import nltk
from nltk.corpus import stopwords

all_stopwords = stopwords.words('english')

cleaned_stop = []
for text in df['clean']:
    
    text_tokens = word_tokenize(text)
    tokens_without_sw =  " ".join([word for word in text_tokens if not word in all_stopwords])
    cleaned_stop.append(tokens_without_sw)

df['clean'] = cleaned_stop

In [11]:
# step 8
# Standartization of verbs
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

clean_verbs = []
for text in df['clean']:
    doc_dep = nlp(text)
    for i in range(len(doc_dep)):
        token = doc_dep[i]
        # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_) 
        if token.pos_== 'VERB':
            try: 
                text = re.sub(token.text, token.lemma_, text)
            except:
                pass
    clean_verbs.append(text)

df['clean'] = clean_verbs

In [52]:
# NOT USED
# step 8.1
# For each sentence, remove the duplicate words
clean = []
for text in df['clean']:
    newText = []
    text_tokens = word_tokenize(text)
    for word in text_tokens:
        if word not in newText:
            newText.append(word)
    clean.append(" ".join(newText))

df['clean'] = clean

In [12]:
# step 9
# Make corpus
# Extract non words
# Correct non words and replace them 
# Make a dictionary non word - corrected word

# Make corpus
corpus = {}
for text in df['clean']:
    for i in word_tokenize(text):
        i = re.sub("[^a-zA-Z]","",i)
        if len(i) > 1:
            corpus[i] = ''
    
# Extract non words
import enchant
d = enchant.Dict("en_US")

non_words = []
for word in corpus.keys():
    if len(word) > 3:
        if d.check(word) is not True:
            non_words.append(word)

In [16]:
# Try splitting words before correcting
splitted = {}
for i in non_words:
    j = infer_spaces(i)
    wordList = j.split(' ')
    if len(wordList) > 2 and d.check(wordList[0]) is True and len(wordList[0])>2 and d.check(wordList[1]) is True and len(wordList[1])>2:
        finalWordList = []
        for word in wordList:
            if len(word) == 1 or d.check(word) is not True: continue
            finalWordList.append(word)
        splitted[i] = ' '.join(finalWordList)

In [19]:
# Split words using the splitted dictionary
clean2 = []
for text in df['clean']:
    newtext = []
    for i in word_tokenize(text):
        i = re.sub("[^a-zA-Z]","",i)
        if i in splitted.keys() and i in non_words:
            newtext.append(splitted[i])
        else:
            newtext.append(i)
    clean2.append(' '.join(newtext))
    
df['clean2'] = clean2

In [25]:
# Recalculate non words
# Make corpus
corpus = {}
for text in df['clean2']:
    for i in word_tokenize(text):
        if len(i) > 1:
            corpus[i] = ''
    
# Extract non words
import enchant
d = enchant.Dict("en_US")
non_words = []
for word in corpus.keys():
    if len(word) > 3:
        if d.check(word) is not True:
            non_words.append(word)
            
# Before: 107 952, after splitting: 94822

In [22]:
# Load autocorrections
with open('autocorrections.pkl', 'rb') as fp:
    autocorrections = pickle.load(fp)
    print(len(autocorrections))

31778


In [23]:
# Autocorrections
clean2 = []
for text in df['clean2']:
    newtext = []
    for i in word_tokenize(text):
        i = re.sub("[^a-zA-Z]","",i)
        if i in non_words and i in autocorrections.keys():
            newtext.append(autocorrections[i])
        else:
            newtext.append(i)
    clean2.append(' '.join(newtext))
    
df['clean2'] = clean2 # 63818 non words

In [15]:
# step # helper function #

from math import log
from nltk.corpus import words

word_list = words.words()

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
words = word_list
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))

maxword = 0
for x in words:
    if len(x) > maxword:
        maxword = len(x)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        j = max(0, i-maxword)
        candidates = enumerate(reversed(cost[j:i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [29]:
# step 10.1

# Remove non words
clean = []
for text in df['clean2']:
    newText = []
    text_tokens = word_tokenize(text)
    for word in text_tokens:
        if d.check(word) is not True:
            continue
        else: newText.append(word)
            
    clean.append(" ".join(newText))
df['base'] = clean

# Remove words with length of 2 or less
clean = []
for text in df['clean2']:
    newText = []
    text_tokens = word_tokenize(text)
    for word in text_tokens:
        if len(word) <= 2:
            continue
        else: newText.append(word)
            
    clean.append(" ".join(newText))
df['base'] = clean

In [36]:
# TRY NOUN STANDARDIZATION OR STEMMING WHICH IS USED IN THE STEP BELOW AFTER CORRECTING WORDS
# Noun Standardization
import enchant
d = enchant.Dict("en_US")

def singular(noun):
    if len(noun) >= 4 and (noun[-3] == 'ses' or noun[-3] == 'xes' or noun[-3] == 'zes'):
        return noun[0:-2]
    elif len(noun) >= 5 and (noun[-4] == 'shes' or noun[-4] == 'ches'):
        return noun[0:-2]          
    elif len(noun) >= 4 and noun[-3] == 'ies':
        noun = noun[0:-3] 
        return noun + 'y'
    elif len(noun) >= 4 and noun[-3] == 'ves':
        noun = noun[0:-3] 
        return noun + 'f'
    elif len(noun) >= 2 and noun[-1] == 's':
        if d.check(noun[0:-1]) is True:
            return noun[0:-1]
        else: return noun
    else: return noun
    
cleaned_nouns = []
for text in df['clean']:
    doc_dep = nlp(text)
    for i in range(len(doc_dep)):
        token = doc_dep[i]
        if token.pos_== 'NOUN':
            text = re.sub(token.text, singular(token.text), text)
            
    cleaned_nouns.append(text)

df['clean_nouns'] = cleaned_nouns

In [37]:
# step 11
# Stemming
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stemmed = []
for text in df['clean']:
    newtext = []
    for i in word_tokenize(text):
        i = re.sub("[^a-zA-Z]","",i)
        newtext.append(porter.stem(i))
    stemmed.append(' '.join(newtext))
    
df['stemmed'] = stemmed

In [39]:
df.to_pickle("preprocessed.pkl")

#### TESTING CORNER

In [104]:
# Check negation

# Antonym replacement

# Check if there are bigrams starting with not
from nltk import word_tokenize 
from nltk.util import ngrams

# step 1, 4, 5, 6, 7.1
negations = []
for text in df['clean']:
    tokens = word_tokenize(text)
    bigrams = list(ngrams(tokens, 2))
    for bigram in bigrams:
        if bigram[0] == 'not' and bigram not in negations:
            negations.append(bigram)

In [108]:
# Replacing negations with antonyms
from nltk.corpus import wordnet
# import nltk
# nltk.download('wordnet')

negations_translated = {}
for antonym in negations:
    antonym = antonym[1]
    
    for syn in wordnet.synsets(antonym):
        for lm in syn.lemmas():
            if lm.antonyms():
                #antonyms.append(lm.antonyms()[0].name())
                negations_translated[antonym] = lm.antonyms()[0].name()

negations_translated

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\draga\AppData\Roaming\nltk_data...


{'change': 'stay',
 'last': 'first',
 'happen': 'dematerialise',
 'get': 'end',
 'sure': 'uncertain',
 'just': 'unfair',
 'believe': 'disbelieve',
 'found': 'lost',
 'prepared': 'unprepared',
 'shut': 'open',
 'have': 'refuse',
 'bring': 'take_away',
 'good': 'ill',
 'be': 'differ',
 'think': 'forget',
 'go': 'no-go',
 'lead': 'follow',
 'worried': 'reassure',
 'rising': 'falling',
 'being': 'differ',
 'give': 'starve',
 'trust': 'mistrust',
 'survive': 'succumb',
 'make': 'break',
 'cool': 'warm',
 'ignore': 'know',
 'laugh': 'cry',
 'all': 'partly',
 'miss': 'hit',
 'worry': 'reassure',
 'exacerbate': 'better',
 'agree': 'disagree',
 'earned': 'unearned',
 'waste': 'conserve',
 'got': 'end',
 'stop': 'begin',
 'taking': 'disclaim',
 'end': 'begin',
 'keeping': 'break',
 'even': 'uneven',
 'real': 'insubstantial',
 'always': 'never',
 'going': 'stop',
 'allow': 'deny',
 'forget': 'mind',
 'doing': 'unmake',
 'ready': 'unready',
 'true': 'untruthful',
 'high': 'low',
 'know': 'ignore',

In [110]:
import pickle

with open('negations_dictionary.pkl', 'wb') as f:
    pickle.dump(negations_translated, f)
        
with open('negations_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [72]:
# Slang Detection and Transformation 

import requests
from bs4 import BeautifulSoup

base = "https://www.internetslang.com/list.asp?i=all&ezpage="

slang_dictionary = {}
for i in range(1, 26):
    URL = base + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    
    for result in soup.findAll("table")[7].find("tbody").find_all("tr")[6:]:
        try:
            slang = result.find_all("td")[0].find_all("a")[0].get_text()
            translation = result.find_all("td")[1].text.strip()

            slang_dictionary[slang] = translation
        except: pass

In [75]:
import pickle

with open('slang_dictionary.pkl', 'wb') as f:
    pickle.dump(slang_dictionary, f)
        
with open('slang_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
    
loaded_dict

{'#': 'Twitter hashtagNumber',
 '#1': 'Number one',
 '$': 'Dollar, money',
 '$_$': 'Has money',
 '%)': 'Drunk, giddy',
 '&': "AmpersandRepresents The Word 'And'",
 '&APOS;S': "Wrongly displayed 's",
 '><': 'Frustration',
 '>,<': 'Angry, annoyed',
 '>.>': 'Suspicious, wary',
 '>.<': 'Frustrated, angry, upset, in pain',
 '>//<': 'Embarrassed, blushing',
 '>3': 'Evil but happyHate (opposite of <3)',
 '>:(': 'Angry',
 '>:)': 'Evil Grin',
 '>:D': 'SchemingManiacal',
 '>:D<': 'Hug',
 '>:O': 'Angry',
 '>O<': 'Yelling',
 '>_>': 'Looking left, shifty look',
 '>_<': 'Frustrated',
 '&HEARTS;': 'HTML code for a heart',
 '<#': 'Misspelling of <3',
 '<>': 'Not equalNo comment',
 '<><': 'Fish',
 '<.<': 'Shifty look',
 '</3': 'Broken Hearted',
 '<2': 'Not quite love (<3)',
 '<3': 'Love (heart shape)',
 '<333': 'Multiple hearts',
 '<3U': 'Love You',
 '<4': 'More than love (<3)',
 '<B': 'Hate',
 '<G>': 'Grin',
 '<_<': 'Sarcasm, look left',
 "'98": '1998',
 "'EM": 'Them',
 '(!)': 'Sarcasm',
 "('_')": 'Em